In [12]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display
import pandas as pd
import json
from pprint import pprint
from google.colab import drive
import re
from urllib.parse import urljoin
import os
from html import unescape
import aiohttp

In [2]:
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i chrome.deb
!apt-get -f install -y

Selecting previously unselected package google-chrome-stable.
(Reading database ... 125080 files and directories currently installed.)
Preparing to unpack chrome.deb ...
Unpacking google-chrome-stable (142.0.7444.59-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Errors were encountered while processing:
 google-chrome-stable
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  libvulkan1 mesa-vulkan-drivers
0 upgraded, 2 newly installed, 0 to remove

In [3]:
!wget -q https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/134.0.6998.165/linux64/chromedriver-linux64.zip
!unzip -q chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

In [4]:
!pip install selenium > /dev/null

In [5]:
!pip install -q playwright nest_asyncio
!playwright install chromium

import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
nest_asyncio.apply()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 5.1 MB/s eta 0:00:00
173.7 MiB [] 0% 0.0s173.7 MiB [] 0% 42.7s173.7 MiB [] 0% 44.0s173.7 MiB [] 0% 31.0s173.7 MiB [] 0% 24.7s173.7 MiB [] 0% 17.9s173.7 MiB [] 0% 13.1s173.7 MiB [] 1% 10.4s173.7 MiB [] 1% 9.4s173.7 MiB [] 1% 8.8s173.7 MiB [] 1% 8.5s173.7 MiB [] 2% 6.4s173.7 MiB [] 3% 5.4s173.7 MiB [] 4% 5.0s173.7 MiB [] 4% 4.8s173.7 MiB [] 4% 4.9s173.7 MiB [] 5% 4.9s173.7 MiB [] 5% 4.5s173.7 MiB [] 6% 4.4s173.7 MiB [] 7% 4.1s173.7 MiB [] 8% 3.9s173.7 MiB [] 8% 3.8s173.7 MiB [] 9% 3.5s173.7 MiB [] 10% 3.6s173.7 MiB [] 11% 3.4s173.7 MiB [] 11% 3.2s173.7 MiB [] 12% 3.2s173.7 MiB [] 12% 3.1s173.7 MiB [] 13% 3.0s173.7 MiB [] 14% 2.9s173.7 MiB [] 15% 2.7s173.7 MiB [] 16% 2.6s173.7 MiB [] 17% 2.6s173.7 MiB [] 18% 2.6s173.7 MiB [] 19% 2.5s173.7 MiB [] 19% 2.4s173.7 MiB [] 20% 2.4s173.7 MiB [] 20% 2.5s173.7 MiB [] 21% 2.5s173.7 MiB [] 22% 2.4s173.7 MiB [] 23% 2.3s173.7 MiB [] 24% 2.2s173.7 MiB [] 25% 2.1s173.7 MiB [] 26% 2.1s173.7 MiB [] 2

In [16]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

nest_asyncio.apply()

async def crawl_kurly_ramen():
    base_url = "https://www.kurly.com"
    page_urls = [
        "https://www.kurly.com/search?sword=%EB%B4%89%EC%A7%80%EB%9D%BC%EB%A9%B4&page=1",
        "https://www.kurly.com/search?sword=%EB%B4%89%EC%A7%80%EB%9D%BC%EB%A9%B4&page=2"
    ]
    results = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for page_idx, url in enumerate(page_urls, start=1):
            print(f"[INFO] 페이지 {page_idx} 로딩 중: {url}")
            await page.goto(url, wait_until="domcontentloaded")
            await page.wait_for_timeout(5000)

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")

            a_tags = soup.find_all("a", class_="css-8bebpy e1c07x488")
            print(f"[DEBUG] 상품 a 태그 개수 (페이지 {page_idx}): {len(a_tags)}")

            for idx, a_tag in enumerate(a_tags, start=1):
                name_tag = a_tag.find("span", class_="css-1dry2r1 e1c07x485")
                name = name_tag.text.strip() if name_tag else "(상품명 없음)"
                full_url = base_url + a_tag.get("href", "")
                results.append({
                    "page": page_idx,
                    "name": name,
                    "url": full_url
                })
                print(f"[{page_idx}-{idx}] {name} → {full_url}")

        await browser.close()

    return results


async def main():
    global data
    data = await crawl_kurly_ramen()
    print("\n✅ 크롤링 완료!")
    print(f"총 {len(data)}개 상품 수집됨 (1+2페이지 합계)\n")
    print(data)  # 👈 리스트 전체 확인

await main()


[INFO] 페이지 1 로딩 중: https://www.kurly.com/search?sword=%EB%B4%89%EC%A7%80%EB%9D%BC%EB%A9%B4&page=1
[DEBUG] 상품 a 태그 개수 (페이지 1): 96
[1-1] [골라담기] 오뚜기 라면 8종 균일가 (택3) → https://www.kurly.com/goods/1001143756
[1-2] [오뚜기] 김치라면 (120GX5) → https://www.kurly.com/goods/1000465961
[1-3] [농심] 신라면 멀티 5입 → https://www.kurly.com/goods/5069267
[1-4] [삼양] 맵탱 마늘조개라면 4입 → https://www.kurly.com/goods/1000336866
[1-5] [삼양] 삼양라면 5입 → https://www.kurly.com/goods/5081455
[1-6] [팔도] 왕뚜껑 봉지라면 4입 → https://www.kurly.com/goods/1001429484
[1-7] [농심] 올리브 짜파게티 5입 → https://www.kurly.com/goods/5061319
[1-8] [오뚜기] 열라면 5입 → https://www.kurly.com/goods/5100805
[1-9] [농심] 농심라면 4입 → https://www.kurly.com/goods/1001085694
[1-10] [팔도]남자라면 4입 → https://www.kurly.com/goods/1000467360
[1-11] [오뚜기] 북엇국라면 4입 → https://www.kurly.com/goods/1000154396
[1-12] [팔도] 틈새라면 5입 → https://www.kurly.com/goods/5095001
[1-13] [농심] 얼큰한 너구리 5입 → https://www.kurly.com/goods/5061318
[1-14] [오뚜기] 마열라면 4입 → https://www.kurly.com/goods/1000358330
[1-1

In [ ]:
#img class="css-1d3w5wq es6jciw0"
#img class = "ktx_yjgr948"

In [17]:
nest_asyncio.apply()

SAVE_DIR = "data/images"
os.makedirs(SAVE_DIR, exist_ok=True)

async def download_image(session, img_url, filename):
    """이미지 다운로드 (비동기)"""
    try:
        async with session.get(img_url) as response:
            if response.status != 200:
                print(f"[WARN] 이미지 요청 실패: {img_url}")
                return
            img_bytes = await response.read()
            with open(filename, "wb") as f:
                f.write(img_bytes)
            print(f"[IMG] 저장 완료: {filename}")
    except Exception as e:
        print(f"[ERROR] 이미지 다운로드 실패: {e}")

async def crawl_and_save_images(product_list):
    """각 상품 페이지에서 이미지 추출 + 저장"""
    base_url = "https://www.kurly.com"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        async with aiohttp.ClientSession() as session:
            for idx, product in enumerate(product_list, start=1):
                url = product["url"]
                name = product["name"].replace("/", "_").replace(" ", "")
                print(f"\n[{idx}] 상품 로딩 중: {url}")

                await page.goto(url, wait_until="domcontentloaded")
                await page.wait_for_timeout(5000)

                html = await page.content()
                soup = BeautifulSoup(html, "html.parser")

                # 🔍 이미지 태그 찾기 (2가지 class 기준)
                img_tags = soup.find_all("img", class_=["css-1d3w5wq es6jciw0", "ktx_yjgr948"])
                print(f"[DEBUG] 이미지 태그 개수: {len(img_tags)}")

                for i, img in enumerate(img_tags, start=1):
                    img_url = img.get("src")
                    if not img_url:
                        continue
                    if img_url.startswith("//"):
                        img_url = "https:" + img_url
                    elif img_url.startswith("/"):
                        img_url = base_url + img_url

                    filename = os.path.join(SAVE_DIR, f"{name}_{i}.jpg")
                    await download_image(session, img_url, filename)

        await browser.close()

# 🧠 실행
await crawl_and_save_images(data)


[1] 상품 로딩 중: https://www.kurly.com/goods/1001143756
[DEBUG] 이미지 태그 개수: 1
[IMG] 저장 완료: data/images/[골라담기]오뚜기라면8종균일가(택3)_1.jpg

[2] 상품 로딩 중: https://www.kurly.com/goods/1000465961
[DEBUG] 이미지 태그 개수: 1
[IMG] 저장 완료: data/images/[오뚜기]김치라면(120GX5)_1.jpg

[3] 상품 로딩 중: https://www.kurly.com/goods/5069267
[DEBUG] 이미지 태그 개수: 1
[IMG] 저장 완료: data/images/[농심]신라면멀티5입_1.jpg

[4] 상품 로딩 중: https://www.kurly.com/goods/1000336866
[DEBUG] 이미지 태그 개수: 1
[IMG] 저장 완료: data/images/[삼양]맵탱마늘조개라면4입_1.jpg

[5] 상품 로딩 중: https://www.kurly.com/goods/5081455
[DEBUG] 이미지 태그 개수: 1
[IMG] 저장 완료: data/images/[삼양]삼양라면5입_1.jpg

[6] 상품 로딩 중: https://www.kurly.com/goods/1001429484
[DEBUG] 이미지 태그 개수: 2
[IMG] 저장 완료: data/images/[팔도]왕뚜껑봉지라면4입_1.jpg
[IMG] 저장 완료: data/images/[팔도]왕뚜껑봉지라면4입_2.jpg

[7] 상품 로딩 중: https://www.kurly.com/goods/5061319
[DEBUG] 이미지 태그 개수: 1
[IMG] 저장 완료: data/images/[농심]올리브짜파게티5입_1.jpg

[8] 상품 로딩 중: https://www.kurly.com/goods/5100805
[DEBUG] 이미지 태그 개수: 1
[IMG] 저장 완료: data/images/[오뚜기]열라면5입_1.jpg

[9] 상품 로딩 

In [19]:
single_products = []
multi_products = []

for item in data:
    name = item["name"]
    if any(k in name for k in ["골라담기", "택", "세트", "모음", "묶음", "구성", "종"]):
        multi_products.append(item)
    else:
        single_products.append(item)

print("단일상품(",len(single_products),")", single_products)
print("복수상품(",len(multi_products),")", multi_products)

단일상품( 169 ) [{'page': 1, 'name': '[오뚜기] 김치라면 (120GX5)', 'url': 'https://www.kurly.com/goods/1000465961'}, {'page': 1, 'name': '[농심] 신라면 멀티 5입', 'url': 'https://www.kurly.com/goods/5069267'}, {'page': 1, 'name': '[삼양] 맵탱 마늘조개라면 4입', 'url': 'https://www.kurly.com/goods/1000336866'}, {'page': 1, 'name': '[삼양] 삼양라면 5입', 'url': 'https://www.kurly.com/goods/5081455'}, {'page': 1, 'name': '[팔도] 왕뚜껑 봉지라면 4입', 'url': 'https://www.kurly.com/goods/1001429484'}, {'page': 1, 'name': '[농심] 올리브 짜파게티 5입', 'url': 'https://www.kurly.com/goods/5061319'}, {'page': 1, 'name': '[오뚜기] 열라면 5입', 'url': 'https://www.kurly.com/goods/5100805'}, {'page': 1, 'name': '[농심] 농심라면 4입', 'url': 'https://www.kurly.com/goods/1001085694'}, {'page': 1, 'name': '[팔도]남자라면 4입', 'url': 'https://www.kurly.com/goods/1000467360'}, {'page': 1, 'name': '[오뚜기] 북엇국라면 4입', 'url': 'https://www.kurly.com/goods/1000154396'}, {'page': 1, 'name': '[팔도] 틈새라면 5입', 'url': 'https://www.kurly.com/goods/5095001'}, {'page': 1, 'name': '[농심] 얼큰한 너구리

In [29]:
with open("single_products.json", "w", encoding="utf-8") as f:
    json.dump(single_products, f, ensure_ascii=False, indent=2)

In [28]:
with open("multi_products.json", "w", encoding="utf-8") as f:
    json.dump(multi_products, f, ensure_ascii=False, indent=2)